In [2]:
import os
import numpy as np
import nibabel as nib

In [2]:
def gather_view(root, view_type, grade):
    scans = []
    count = 0
    for GG in os.listdir(root):
        if GG == grade:
            GG_dir = os.path.join(root, GG)
            if os.path.isdir(GG_dir):
                for patient in os.listdir(GG_dir):
                    patient_dir = os.path.join(GG_dir, patient)
                    if os.path.isdir(patient_dir):
                        for patient_files in os.listdir(patient_dir):
                            if view_type in patient_files:
                                scan_root = os.path.join(patient_dir, patient_files)
                                img = nib.load(scan_root)
                                scans.append(img.get_fdata())
                                count += 1
                                if count%10 == 0:
                                    print(count)
                
    return np.asarray(scans).astype(np.uint8)


RUN ONE AT A TIME OR ELSE MEMORY ERROR AND YOURE DOOMED

In [ ]:
data_path = 'data/MICCAI_BraTS_2018_Data_Training'
seg_scans_hgg = gather_view(data_path, 'seg.nii.gz', 'HGG')
seg_scans_lgg = gather_view(data_path, 'seg.nii.gz', 'LGG')
seg_scans = np.concatenate((seg_scans_hgg, seg_scans_lgg), axis=0)

In [ ]:
data_path = 'data/MICCAI_BraTS_2018_Data_Validation'
flair_scans_hgg = gather_view(data_path, 'flair.nii.gz', 'HGG')
flair_scans_lgg = gather_view(data_path, 'flair.nii.gz', 'LGG')
flair_scans = np.concatenate((flair_scans_hgg, flair_scans_lgg), axis=0)

In [ ]:
data_path = 'data/MICCAI_BraTS_2018_Data_Validation'
t1_scans_hgg = gather_view(data_path, 't1.nii.gz', 'HGG')
t1_scans_lgg = gather_view(data_path, 't1.nii.gz', 'LGG')
t1_scans = np.concatenate((t1_scans_hgg, t1_scans_lgg), axis=0)

In [ ]:
data_path = 'data/MICCAI_BraTS_2018_Data_Validation'
t1ce_scans_hgg = gather_view(data_path, 't1ce.nii.gz', 'HGG')
t1ce_scans_lgg = gather_view(data_path, 't1ce.nii.gz', 'LGG')
t1ce_scans = np.concatenate((t1ce_scans_hgg, t1ce_scans_lgg), axis=0)

In [ ]:
data_path = 'data/MICCAI_BraTS_2018_Data_Training'
t2_scans_hgg = gather_view(data_path, 't2.nii.gz', 'HGG')
t2_scans_lgg = gather_view(data_path, 't2.nii.gz', 'LGG')
t2_scans = np.concatenate((t2_scans_hgg, t2_scans_lgg), axis=0)

In [ ]:
import pickle
with open("data/pkl_files/flair_scans.pkl", "rb") as fp:
    flair_scans = pickle.load(fp)
with open("data/pkl_files/t1_scans.pkl", "rb") as fp:
    t1_scans = pickle.load(fp)
with open("data/pkl_files/t2_scans.pkl", "rb") as fp:
    t2_scans = pickle.load(fp)
with open("data/pkl_files/t1ce_scans.pkl", "rb") as fp:
    t1ce_scans = pickle.load(fp)
with open("data/pkl_files/seg_scans.pkl", "rb") as fp:
    seg_scans = pickle.load(fp)

In [ ]:
flair_scans = flair_scans[:,:,:,30:120:1]
t1_scans = t1_scans[:,:,:,30:120:1]
t2_scans = t2_scans[:,:,:,30:120:1]
t1ce_scans = t1ce_scans[:,:,:,30:120:1]
seg_scans = seg_scans[:,:,:,30:120:1]

In [ ]:
flair_scans = np.transpose(flair_scans, (0,3,1,2))
t1_scans = np.transpose(t1_scans, (0,3,1,2))
t2_scans = np.transpose(t2_scans, (0,3,1,2))
t1ce_scans = np.transpose(t1ce_scans, (0,3,1,2))
seg_scans = np.transpose(seg_scans, (0,3,1,2))

In [ ]:
flair_scans = np.reshape(flair_scans, (25650, 1, 240, 240))
t1_scans = np.reshape(t1_scans, (25650, 1, 240, 240))
t2_scans = np.reshape(t2_scans, (25650, 1, 240, 240))
t1ce_scans = np.reshape(t1ce_scans, (25650, 1, 240, 240))
seg_scans = np.reshape(seg_scans, (25650, 1, 240, 240))

In [ ]:
stacked_inputs = np.concatenate((flair_scans, t1_scans, t1ce_scans, t2_scans), axis=1)

DO NOT RUN THE OPEN PICKLE FILES AGAIN.

In [ ]:
with open("data/pkl_files/prelim_inputs_0.pkl", "wb") as fp:
    pickle.dump(stacked_inputs[0:12825:1,:,:,:], fp)

In [ ]:
with open("data/pkl_files/prelim_inputs_1.pkl", "wb") as fp:
    pickle.dump(stacked_inputs[12825:,:,:,:], fp)

In [ ]:
with open("data/pkl_files/prelim_outputs.pkl", "wb") as fp:
    pickle.dump(seg_scans, fp)

Plotting to see the views

In [ ]:
import matplotlib.pyplot as plt

# idx = np.random.randint(0,25650)
idx = 13756
seg_img = seg_scans[idx,0,:,:]

fig = plt.figure()
plt.title('Ground Truth Segmented Tumor')
plt.imshow(seg_img)
plt.show()
# fig.savefig('example_segmented_ground_truth.png')


fig = plt.figure()
plt.title('FLAIR View')
flair_img = flair_scans[idx,0,:,:]
plt.imshow(flair_img)
plt.show()
# fig.savefig('example_flair_view.png')

fig = plt.figure()
plt.title('T1 View')
t1_img = t1_scans[idx,0,:,:]
plt.imshow(t1_img)
plt.show()
# fig.savefig('example_t1_view.png')

fig = plt.figure()
plt.title('T2 View')
t2_img = t2_scans[idx,0,:,:]
plt.imshow(t2_img)
plt.show()
# fig.savefig('example_t2_view.png')

fig = plt.figure()
plt.title('T1ce view')
t1ce_img = t1ce_scans[idx,0,:,:]
plt.imshow(t1ce_img)
plt.show()
# fig.savefig('example_t1ce_view.png')

Truncating Images to 30x30

In [3]:
import pickle
with open("data/pkl_files/prelim_inputs_0.pkl", "rb") as fp:
    prelim_inputs_0 = pickle.load(fp)
with open("data/pkl_files/prelim_inputs_1.pkl", "rb") as fp:
    prelim_inputs_1 = pickle.load(fp)
with open("data/pkl_files/prelim_outputs.pkl", "rb") as fp:
    outputs = pickle.load(fp)

In [4]:
inputs = np.concatenate((prelim_inputs_0, prelim_inputs_1), axis=0)

In [5]:
sampled_inputs = np.zeros((inputs.shape[0], inputs.shape[1], 32, 32))
sampled_outputs = np.zeros((outputs.shape[0], outputs.shape[1], 32, 32))
# numpy.random.randint chooses a value from 0 to high_value, non_inclusive.
rand_x0_cutoffs = np.random.randint(low=50, high=170, size=inputs.shape[0])
rand_y0_cutoffs = np.random.randint(low=50, high=170, size=inputs.shape[0])
for idx in range(inputs.shape[0]):
    x0 = rand_x0_cutoffs[idx]
    y0 = rand_y0_cutoffs[idx]
    sampled_inputs[idx] = inputs[idx, : , x0:x0+32, y0:y0+32]
    sampled_outputs[idx] = outputs[idx, :, x0:x0+32, y0:y0+32]

In [6]:
# Sanity check
print(inputs.shape)
print(outputs.shape)
print(sampled_inputs.shape)
print(sampled_outputs.shape)

(25650, 4, 240, 240)
(25650, 1, 240, 240)
(25650, 4, 32, 32)
(25650, 1, 32, 32)


In [10]:
with open("data/pkl_files/trimmed_inputs.pkl", "wb") as fp:
    pickle.dump(sampled_inputs, fp)
with open("data/pkl_files/trimmed_outputs.pkl", "wb") as fp:
    pickle.dump(sampled_outputs, fp)

In [12]:
with open("data/pkl_files/trimmed_inputs.pkl", "rb") as fp:
    test_inputs = pickle.load(fp)
with open("data/pkl_files/trimmed_outputs.pkl", "rb") as fp:
    test_outputs = pickle.load(fp)
print(test_inputs.shape)
print(test_outputs.shape)

(25650, 4, 32, 32)
(25650, 1, 32, 32)


Only using 3 views

In [3]:
import pickle
with open("data/pkl_files/flair_scans.pkl", "rb") as fp:
    flair_scans = pickle.load(fp)
with open("data/pkl_files/t1_scans.pkl", "rb") as fp:
    t1_scans = pickle.load(fp)
with open("data/pkl_files/t2_scans.pkl", "rb") as fp:
    t2_scans = pickle.load(fp)

In [4]:
flair_scans = flair_scans[:,:,:,30:120:1]
t1_scans = t1_scans[:,:,:,30:120:1]
t2_scans = t2_scans[:,:,:,30:120:1]

In [5]:
flair_scans = np.reshape(flair_scans, (25650, 1, 240, 240))
t1_scans = np.reshape(t1_scans, (25650, 1, 240, 240))
t2_scans = np.reshape(t2_scans, (25650, 1, 240, 240))

In [6]:
stacked_inputs = np.concatenate((flair_scans, t1_scans, t2_scans), axis=1)

In [9]:
with open("data/pkl_files/prelim_outputs.pkl", "rb") as fp:
    outputs = pickle.load(fp)

In [12]:
sampled_inputs = np.zeros((stacked_inputs.shape[0], stacked_inputs.shape[1], 32, 32))
sampled_outputs = np.zeros((outputs.shape[0], outputs.shape[1], 32, 32))
# numpy.random.randint chooses a value from 0 to high_value, non_inclusive.
rand_x0_cutoffs = np.random.randint(low=50, high=170, size=stacked_inputs.shape[0])
rand_y0_cutoffs = np.random.randint(low=50, high=170, size=stacked_inputs.shape[0])
for idx in range(stacked_inputs.shape[0]):
    x0 = rand_x0_cutoffs[idx]
    y0 = rand_y0_cutoffs[idx]
    sampled_inputs[idx] = stacked_inputs[idx, : , x0:x0+32, y0:y0+32]
    sampled_outputs[idx] = outputs[idx, :, x0:x0+32, y0:y0+32]

In [10]:
print(outputs.shape)

(25650, 1, 240, 240)


In [11]:
print(stacked_inputs.shape)

(25650, 3, 240, 240)


In [13]:
print(sampled_inputs.shape)

(25650, 3, 32, 32)


In [14]:
print(sampled_outputs.shape)

(25650, 1, 32, 32)


In [15]:
with open("data/pkl_files/3_chan_trimmed_inputs.pkl", "wb") as fp:
    pickle.dump(sampled_inputs, fp)
with open("data/pkl_files/3_chan_trimmed_outputs.pkl", "wb") as fp:
    pickle.dump(sampled_outputs, fp)

In [17]:
with open("data/pkl_files/3_chan_trimmed_inputs.pkl", "rb") as fp:
    test_inputs = pickle.load(fp)
with open("data/pkl_files/3_chan_trimmed_outputs.pkl", "rb") as fp:
    test_outputs = pickle.load(fp)
print(test_inputs.shape)
print(test_outputs.shape)

(25650, 3, 32, 32)
(25650, 1, 32, 32)
